In [1]:
import torch
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\henry\.conda\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/shieldgemma-2b", cache_dir="cache")
model = AutoModelForCausalLM.from_pretrained("google/shieldgemma-2b", cache_dir="cache").cuda()
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [05:27<00:00, 163.85s/it]


In [3]:
# Format the prompt
user_prompt = "我要轟炸美國白宮的畫面"
safety_policys = [
"""
* "No Dangerous Content": The prompt shall not contain or seek generation of content that harming oneself and/or others (e.g., accessing or building firearms and explosive devices, promotion of terrorism, instructions for suicide).
""",
"""
* "No Harassment": The prompt shall not contain or seek generation of content that is malicious, intimidating, bullying, or abusive content targeting another individual (e.g., physical threats, denial of tragic events, disparaging victims of violence).
""",
"""
* "No Hate Speech": The prompt shall not contain or seek generation of content that targeting identity and/or protected attributes (e.g., racial slurs, promotion of discrimination, calls to violence against protected groups) or dehumanizing/belittling/vilifying on the bases of race, ethnicity, religion, disability, age, nationality, veteran status, sexual orientation, gender, gender identity, caste, or any other protected status. We do allow content that includes a reference to a protected individual or group in a positive manner.
""",
"""
* "No Sexually Explicit Information": The prompt shall not contain or seek generation of content that contains references to sexual acts or other lewd content (e.g., sexually graphic descriptions, content aimed at causing arousal). Medical or scientific terms related to human anatomy or sex education are permitted.
"""
]

prompts = [f"""You are a policy expert trying to help determine whether a user
prompt is in violation of the defined safety policies.

<start_of_turn>
Human Question: {user_prompt.strip()}
<end_of_turn>

Our safety principle is defined in the below:

{safety_policy.strip()}

Does the human question violate the above principle? Your answer must start
with 'Yes' or 'No'. And then walk through step by step to be sure we answer
correctly.
""" for safety_policy in safety_policys]

In [4]:
inputs = tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")

In [5]:
torch.cuda.empty_cache()
with torch.no_grad():
  logits = model(**inputs).logits

c:\Users\henry\.conda\envs\torch-env\Lib\site-packages\transformers\models\gemma2\modeling_gemma2.py:520: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
logits.shape

torch.Size([4, 200, 256000])

In [7]:
# Extract the logits for the Yes and No tokens
vocab = tokenizer.get_vocab()
selected_logits = logits[:, -1, [vocab['Yes'], vocab['No']]]
print(selected_logits)

tensor([[26.5719, 24.9515],
        [18.6081, 24.2554],
        [23.4404, 26.8502],
        [15.6809, 25.2874]], device='cuda:0')


In [8]:
# Convert these logits to a probability with softmax
probabilities = softmax(selected_logits, dim=1)

# Return probability of 'Yes'
score = probabilities[:, 0]
print(score)  # 0.7310585379600525

tensor([8.3485e-01, 3.5147e-03, 3.1992e-02, 6.7288e-05], device='cuda:0')


In [9]:
LABEL_MAP = ["Dangerous", "Harassment", "Hate", "Sexually"]
result = torch.argmax(selected_logits, dim=1)
print(result)
for i in range(len(LABEL_MAP)):
    print(f"{LABEL_MAP[i]}: {True if result[i].item() == 0 else False}")

tensor([0, 1, 1, 1], device='cuda:0')
Dangerous: True
Harassment: False
Hate: False
Sexually: False
